# CW1 — Task 2: Create MySQL DB (Code) + Import CSV + SQL Query

- **Name:** Akia Hans Swin Carreon
- **Student ID UoR:** *carreona@roehampton.ac.uk*
- **Student ID Lithan:** *LS07432@learning.lithan.com*

Database created via code: `module15_cw1_de_py`  
Table: `books_import_py`


## Install Dependencies (If needed)


In [ ]:
%pip install -q pandas mysql-connector-python

## Load `.env` File & Config


In [ ]:
import os
from pathlib import Path

# Notebook runs from /notebooks, so .env is in the parent (project root)
ENV_PATH = (Path.cwd() / ".." / ".env").resolve()

print("CWD:", Path.cwd().resolve())
print("Trying to load .env from:", ENV_PATH, "exists:", ENV_PATH.exists())

if ENV_PATH.exists():
    for line in ENV_PATH.read_text(encoding="utf-8").splitlines():
        line = line.strip()
        if not line or line.startswith("#") or "=" not in line:
            continue
        k, v = line.split("=", 1)
        os.environ.setdefault(k.strip(), v.strip())
else:
    print("WARNING: .env not found; using defaults")

DB_HOST = os.getenv("DB_HOST", "127.0.0.1")
DB_PORT = int(os.getenv("DB_PORT", "3306"))
DB_USER = os.getenv("DB_USER", "root")
DB_PASSWORD = os.getenv("DB_PASSWORD", "")

DB_NAME = "module15_cw1_de_py"
TABLE_NAME = "books_import_py"
CSV_PATH = (Path.cwd() / "../data/processed/books.csv").resolve()

print("Host:", DB_HOST)
print("Port:", DB_PORT)
print("User:", DB_USER)
print("Target DB:", DB_NAME)
print("Table:", TABLE_NAME)
print("CSV:", CSV_PATH, "exists:", CSV_PATH.exists())


## Create DataBase & Table in Code


In [ ]:
import re
import mysql.connector

IDENT_RE = re.compile(r"^[A-Za-z0-9_]+$")

def validate_ident(name: str, kind: str) -> str:
    if not IDENT_RE.match(name):
        raise ValueError(f"Invalid {kind}: {name!r} (use only letters, numbers, underscore)")
    return name

validate_ident(DB_NAME, "database")
validate_ident(TABLE_NAME, "table")

# Connect without selecting DB
conn = mysql.connector.connect(
    host=DB_HOST,
    port=DB_PORT,
    user=DB_USER,
    password=DB_PASSWORD,
)
cur = conn.cursor()

cur.execute(f"CREATE DATABASE IF NOT EXISTS `{DB_NAME}` CHARACTER SET utf8mb4 COLLATE utf8mb4_general_ci;")
print("[OK] ensured DB exists:", DB_NAME)

cur.execute(f"""
CREATE TABLE IF NOT EXISTS `{DB_NAME}`.`{TABLE_NAME}` (
  title VARCHAR(255) NOT NULL,
  authors TEXT NOT NULL,
  year INT NOT NULL,
  star_rating FLOAT NULL,
  price DECIMAL(10,2) NOT NULL,
  source_url TEXT NOT NULL
);
""")
print("[OK] ensured table exists:", f"{DB_NAME}.{TABLE_NAME}")

conn.commit()
cur.close()
conn.close()


## Import CSV into MySQL (Handles Numpy Types)


In [ ]:
import pandas as pd
import mysql.connector

df = pd.read_csv(CSV_PATH)

required_cols = ["title", "authors", "year", "star_rating", "price", "source_url"]
missing = [c for c in required_cols if c not in df.columns]
if missing:
    raise ValueError(f"CSV missing required columns: {missing}")

# Clean types
df["year"] = pd.to_numeric(df["year"], errors="coerce")
df["star_rating"] = pd.to_numeric(df["star_rating"], errors="coerce")
df["price"] = pd.to_numeric(df["price"], errors="coerce")

df = df.dropna(subset=["title", "authors", "year", "price", "source_url"]).copy()

def to_py(v):
    if pd.isna(v):
        return None
    if hasattr(v, "item"):
        try:
            return v.item()
        except Exception:
            pass
    return v

rows = [
    tuple(to_py(v) for v in row)
    for row in df[required_cols].itertuples(index=False, name=None)
]

conn = mysql.connector.connect(
    host=DB_HOST,
    port=DB_PORT,
    user=DB_USER,
    password=DB_PASSWORD,
    database=DB_NAME,
)
cur = conn.cursor()

cur.execute(f"TRUNCATE TABLE `{TABLE_NAME}`;")

insert_sql = f"""
INSERT INTO `{TABLE_NAME}` (title, authors, year, star_rating, price, source_url)
VALUES (%s, %s, %s, %s, %s, %s)
"""
cur.executemany(insert_sql, rows)
conn.commit()

cur.execute(f"SELECT COUNT(*) FROM `{TABLE_NAME}`;")
count = cur.fetchone()[0]
print(f"[OK] Inserted rows: {count}")

cur.close()
conn.close()


## Required SQL Query


In [ ]:
import mysql.connector
import pandas as pd

conn = mysql.connector.connect(
    host=DB_HOST,
    port=DB_PORT,
    user=DB_USER,
    password=DB_PASSWORD,
    database=DB_NAME,
)

query = f"""
SELECT title, authors, price
FROM `{TABLE_NAME}`
ORDER BY price DESC;
"""

df_sql = pd.read_sql(query, conn)
display(df_sql)

df_count = pd.read_sql(f"SELECT COUNT(*) AS row_count FROM `{TABLE_NAME}`;", conn)
display(df_count)

conn.close()
